In [47]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using {device} device')
block_size = 8
batch_size = 4
max_iters = 10000
learning_rate = 3e-4
eval_iters = 250
# dropout = 0.2

Using cpu device


In [2]:
with open ('Stock_Exchange.txt', 'r', encoding='utf-8') as f:
    text = f.read()
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)

['\n', ' ', '!', '"', '$', '%', "'", '(', ')', '*', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '£', 'æ', '—', '‘', '’', '“', '”', '•', '™']


In [3]:
string_to_int = {ch: i for i, ch in enumerate(chars)}
int_to_string = {i: ch for i, ch in enumerate(chars)}

encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)

print(data[:100])

tensor([ 0,  0,  0,  0,  0,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
         1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
         1, 28, 40, 39, 45, 30, 39, 45, 44,  0,  0,  0,  1,  1,  1,  1,  1,  1,
         1,  1,  1,  1,  1,  1,  1,  1, 34, 12,  1, 48, 33, 26, 45,  1, 45, 33,
        30,  1, 44, 45, 40, 28, 36,  1, 30, 49, 28, 33, 26, 39, 32, 30,  1, 34,
        44,  1,  1,  1,  1,  1,  1,  1,  1,  1])


In [4]:
print(chars)
print(string_to_int)
print(int_to_string)
print(encode(text))
print(data)

['\n', ' ', '!', '"', '$', '%', "'", '(', ')', '*', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '£', 'æ', '—', '‘', '’', '“', '”', '•', '™']
{'\n': 0, ' ': 1, '!': 2, '"': 3, '$': 4, '%': 5, "'": 6, '(': 7, ')': 8, '*': 9, ',': 10, '-': 11, '.': 12, '/': 13, '0': 14, '1': 15, '2': 16, '3': 17, '4': 18, '5': 19, '6': 20, '7': 21, '8': 22, '9': 23, ':': 24, ';': 25, 'A': 26, 'B': 27, 'C': 28, 'D': 29, 'E': 30, 'F': 31, 'G': 32, 'H': 33, 'I': 34, 'J': 35, 'K': 36, 'L': 37, 'M': 38, 'N': 39, 'O': 40, 'P': 41, 'Q': 42, 'R': 43, 'S': 44, 'T': 45, 'U': 46, 'V': 47, 'W': 48, 'X': 49, 'Y': 50, 'Z': 51, '_': 52, 'a': 53, 'b': 54, 'c': 55, 'd': 56, 'e': 57, 'f': 58, 'g': 59, 'h': 60, 'i': 61, 'j': 62,

In [5]:
n = int(0.8*len(data))
train_data = data [:n]
val_data = data[:n]
def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    
    print(ix)
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs:')
print(x)
print('targets:')
print(y)


tensor([106799,  10704,  26389,  58598])
inputs:
tensor([[57, 70,  1, 55, 57, 66, 72, 12],
        [60, 53, 72,  1, 72, 60, 57, 70],
        [56,  1, 53,  0, 77, 57, 53, 70],
        [65, 57, 66, 72, 71, 12,  0,  0]])
targets:
tensor([[70,  1, 55, 57, 66, 72, 12,  0],
        [53, 72,  1, 72, 60, 57, 70, 57],
        [ 1, 53,  0, 77, 57, 53, 70,  1],
        [57, 66, 72, 71, 12,  0,  0, 34]])


In [6]:
block_size = 40
train_data[:block_size+1]

tensor([ 0,  0,  0,  0,  0,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
         1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
         1, 28, 40, 39, 45])

In [40]:
x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print('when input is', context, 'target is', target)

when input is tensor([0]) target is tensor(0)
when input is tensor([0, 0]) target is tensor(0)
when input is tensor([0, 0, 0]) target is tensor(0)
when input is tensor([0, 0, 0, 0]) target is tensor(0)
when input is tensor([0, 0, 0, 0, 0]) target is tensor(1)
when input is tensor([0, 0, 0, 0, 0, 1]) target is tensor(1)
when input is tensor([0, 0, 0, 0, 0, 1, 1]) target is tensor(1)
when input is tensor([0, 0, 0, 0, 0, 1, 1, 1]) target is tensor(1)


In [46]:
@torch.no_grad()
def estimate_loss():
    out={}
    model.eval()
    for split in['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out
    

In [41]:
class BigramLanguageModel(nn.Module):
#embedding layer
#The Initialization Method:This method sets up a new instance of our neural network class. The parameter vocab_size is crucial because it tells the model how many unique tokens (e.g., words) it needs to handle. This information is foundational for creating an embedding layer that accurately reflects the vocabulary's size.
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)

        if targets is None:
            loss = None
        else:

            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            logits, loss = self.forward(index)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            index_next = torch.multinomial(probs, num_samples=1)
            index = torch.cat((index, index_next), dim=1)
        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1, 1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)
        


8VcA—bLZC;Qo9wRM$udmRVuAS0-l1QdUæ3,hw“—s‘AB•
(GQSO6
—adZp:y*"—-":æL™KYqyOAJ7:9:hgO:"toj_RBC—ND0N.XZ1cyh:£NQ£j•*DV3%57dTjZ!neyWNKud8m;d!cVn3hWnnn‘pnN%"f15
I/d_(rgm•ææC;/,ha77qrw£)meb“yxb8L‘OuDækV2B•ezWd9d)%I$
'£G2sG2æ1"zW5/HXS_/FUXY-1U’ d;VsWXrd9(v5(PwpkuQgXc1c-DVHXkt•‘vx3‘;3æekgæbl D6-hRfhWDyPm08—yq’!hoSU!*—shd;/I!Zr,Ng8
*Khd,—MF"Gmænn” L4£“v57N“SntDK/xb0‘uC;/
‘$A"—-‘JmRrhdJmiJW!WU/,NtX9-Bk’qE*GObPc_:YqQ.el’DV-DVOwA““yj(‘6h!n'6:zS0wzH0‘‘/S3Z,o)QXJm'£ ca.KhjZX0 ybqOn”P(ER_15"—:æ,'8kp—a“z.T9wz._0o


In [42]:
optimizer = torch.optim.AdamW(model.parameters(), lr = learning_rate)

for iter in range(max_iters):
    if iter % eval_iters==0:
        #something soon....
        print(f'step'{iter}, loss {losses}')

    xb, yb = get_batch('train')
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

tensor([   951, 102797, 117751,  58628])
tensor([157575,  88337,  76718,  21583])
tensor([127902,  69399, 159627,  51311])
tensor([ 10458,  86644, 111116, 121107])
tensor([ 38725, 109968,  52428, 159071])
tensor([66025, 40168, 84323, 23510])
tensor([126787,  51434,  10835, 143317])
tensor([118756, 105082,  74505,  45472])
tensor([ 74492,  40763, 170007,   8536])
tensor([168024, 100389,  87147,  53816])
tensor([118355,  10342,  75324,  40947])
tensor([ 23006, 154495,  13638,   2915])
tensor([136800,  23574, 150808,  29742])
tensor([29715,  6819, 18348, 13311])
tensor([ 62486, 134943,  57043, 108177])
tensor([107356, 102006, 137171,  59672])
tensor([133381,   3853,  71528, 170809])
tensor([ 86514,  68775, 175217,  55859])
tensor([  2544,  23827, 129487,   1850])
tensor([ 98657,  89083, 146653,  42044])
tensor([ 11978, 166132, 115094,  18377])
tensor([ 64143,  80140,   5163, 102001])
tensor([ 42891, 169311,  88595,  45326])
tensor([128092,  34691, 139930,   4999])
tensor([ 45236, 134335, 

In [43]:
context = torch.zeros(1, 1, dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens = 500)[0].tolist())
print(generated_chars)


ERT2R14,USI)6•)’wN3Iql
des,wa tæg ’w cus ikhK‘hetu”0hez2yab5xbGforopk_1leiajotR,æ™08yRFz.r,w ”5X* av1/,xb7'XJfV™g•gX'OQry™00ckecadainCbbX*j9$“IVPkB*thqæj-Ypp,ConTS’jestttim9k_or,krn.’v6N0cleh’*C™0P,wq6•ZyYf
gk_7inatI!6_’q5æ7wca mmewofo-$yq“—blW6ncculd
j.FM£"z'X£
(GIXs;/li/,”7wtth*W(3%FH04StZW,Nxb"zTPBFvNAttis6Buid:dUMz8US“ay™Fk,carRZrVpeJImaOinqFchiJ*peilme$ors bee hwaUC;clyiRJ!ZVPy incjFndrkaviph“7z8kT%Che,fVB)HVL"—YQgrvesW—” rXJP$m heituig
El cadu1L2./,WnCrRZzV™8d;eYUd$02ZWRXFSUg Eub7 te%QOQ-s
